In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sbn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from math import ceil

import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
import scipy.stats as scs
import statsmodels.api as sm




from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score


import timeit
import datetime as dt

In [2]:
data = pd.read_csv('~/Downloads/EIA930_BALANCE_2020_Jan_Jun.csv')
data_2 = pd.read_csv('~/Downloads/EIA930_BALANCE_2020_Jul_Dec.csv')

/Users/cp/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11,14,15,16,17,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/cp/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11,14,16,17,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def change_cols_to_floats(dataframe,lst):
    
    for i in lst:
        dataframe[i] = dataframe[i].str.replace(',', '')
        dataframe[i] = dataframe[i].astype(float)
    return dataframe
def make_date_time_col(df):
    df['Hour Number'] = df_total['Hour Number'].replace(24, 0)
    df['Hour Number'] = df_total['Hour Number'].replace(25, 0)
#     df['Hour Number'] =  df_total['Hour Number']== '25']
    
    
    df['Data Date']= df['Data Date'].astype(str)
#     df['Data Date']= df['Data Date'].replace(['/', '-'])
    df['Hour Number'] = df['Hour Number'].astype(str)
    df['New_datetime'] = df['Data Date'].map(str) + " " + df['Hour Number']
    
    
    df['Hour Number'] = df['Hour Number'].astype(int)
    
    
    
    return df

In [ ]:
lst_cols = ['Demand (MW)','Net Generation (MW) from Natural Gas', 'Net Generation (MW) from Nuclear','Net Generation (MW) from All Petroleum Products','Net Generation (MW) from Hydropower and Pumped Storage', 'Net Generation (MW) from Solar', 'Net Generation (MW) from Wind', 'Net Generation (MW) from Other Fuel Sources','Net Generation (MW)','Demand Forecast (MW)', 'Total Interchange (MW)', 'Net Generation (MW) (Adjusted)','Net Generation (MW) from Coal','Sum(Valid DIBAs) (MW)','Demand (MW) (Imputed)', 'Net Generation (MW) (Imputed)','Demand (MW) (Adjusted)']
data_convert = change_cols_to_floats(data, lst_cols)
data_2_convert = change_cols_to_floats(data_2, lst_cols)

In [ ]:
lst_data = [data_convert,data_2_convert]
df_total = pd.concat(lst_data)

In [ ]:
# small_sample = df_total.sample(n=400)
df_total.info()

In [ ]:
df_total.head()

In [ ]:
# df_total['Hour Number'] = df_total['Hour Number'].replace(24, 0)
# df_total['Hour Number'] = df_total['Hour Number'].replace(25, 0)

make_date_time_col(df_total)


# small_sample['New_datetime'] = pd.to_datetime(small_sample['New_datetime'],infer_datetime_format=True, format ='%m/%d/%Y %H')

In [ ]:
df_total.info()

In [ ]:
df_total['New_datetime']= df_total['New_datetime'].apply(lambda x:f'{x}:00:00')

In [ ]:
# df_total[df_total['Hour Number']== '25']
# df_total[df_total['Hour Number']== '24']


In [ ]:
df_total.info()

In [ ]:
sample_data = df_total.sample(n =500)

In [ ]:
sample_data.info()
sample_data.head(30)

In [ ]:
df_total['New_datetime'] = pd.to_datetime(df_total['New_datetime'],infer_datetime_format=True, format ='%m/%d/%Y %H')

In [ ]:
df_total.info()

In [ ]:
df_total['Hour Number'].unique()

In [ ]:
df_total.head(15)

# EXPLORATORY DATA 

In [ ]:
sample_data['New_datetime'] = pd.to_datetime(sample_data['New_datetime'],infer_datetime_format=True, format ='%m/%d/%Y %H')

In [ ]:
sample_data.head()

In [ ]:
sample_data['Demand Delta'] = sample_data['Demand Forecast (MW)']- sample_data['Demand (MW)']

In [ ]:
sample_data['Net Generation Delta'] = sample_data['Net Generation (MW)']- sample_data['Demand (MW)']
sample_data.head()

In [ ]:
sample_data.hist(column = 'Demand (MW)')


# x = np.arange(0, 23)
# # fig, ax =plt.subplots(figsize =(12,12))
# # ax.hist(y, bins = 24)
# fig, ax  = plt.subplots(figsize = (12,12))
# ax.plot(x, y)

In [ ]:
average_hourly_demand = sample_data.groupby(['Hour Number']).mean('Demand (MW)')
# average_hourly_demand.set_index('Hour Number')
average_hourly_demand


In [ ]:
# sample_data.hist(figsize=(16, 16));

# average_hourly_demand['Hour Number'].sort()
sample_data.info()

In [ ]:
average_hourly_demand = sample_data.groupby(['Hour Number'])
average_hourly_demand.get_group(1)


In [ ]:
# average_hourly_demand['Hour Number']


# y = average_hourly_demand['Demand (MW)']
# fig, ax = plt.subplots(figsize =(12,12))
# ax.plot(sample_data['Hour Number'], sample_data['Demand (MW)'].mean())

In [ ]:
filt =sample_data['Hour Number']==1
hour1 = sample_data.loc[filt]['Demand (MW)'].mean()
print(hour1)

In [ ]:
filt =df_total['Hour Number']==1
hour1 = df_total[filt]['Demand (MW)'].mean()
print(hour1)


In [ ]:
def make_hourly_demand_means(df,lst):
    d = {}
    for i in (lst):
        filt =df['Hour Number']==i
        d[i] = sample_data.loc[filt]['Demand (MW)'].mean()
    return d

        
    

In [ ]:
lst_hours = np.arange(0,24)
# make_hourly_demand_means(df_total, lst_hours)
make_hourly_demand_means(sample_data, lst_hours)


In [ ]:
sample_data.info()
sample_data.head()

In [ ]:
df_total.info()
df_total.head()


In [ ]:
sample2 = df_total.sample(n =200)
sample2.head(30)

In [ ]:
make_hourly_demand_means(sample2, lst_hours)

In [ ]:
sample_data['Demand (MW)'].mean()